In [ ]:
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pathlib as pl
import squidpy as sq

from typing import Tuple
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import infercnvpy as cnv

In [ ]:
from statannotations.Annotator import Annotator

In [ ]:
import pathlib as pl

In [ ]:
def pretty_ax(ax):
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.tick_params(
        axis='both',  
        which='both',      
        bottom=True,     
        top=False,
        left=False,
        labelbottom=True,
        labelleft = True)
    ax.spines["bottom"].set_linewidth(1.5)
    ax.spines["left"].set_linewidth(1.5)

In [ ]:
def get_preprocessed_sample(sample_path: pl.Path, min_counts: int, pct_mt: int, min_cells: int) -> sc.AnnData:

    adata = sc.read_visium(path=sample_path)

    adata.var_names_make_unique()
    adata.var["mt"] = adata.var_names.str.startswith("MT-")
    sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"], inplace=True)

    adata.obsm["spatial"] = adata.obsm["spatial"].astype(int)

    sc.pp.filter_cells(adata, min_counts=min_counts)
    adata = adata[adata.obs["pct_counts_mt"] < pct_mt]
    print(f"#cells after MT filter: {adata.n_obs}")
    sc.pp.filter_genes(adata, min_cells=min_cells)
    
    return adata

In [ ]:
def filter_genes(data: np.ndarray, min_cells: int) -> Tuple[np.ndarray, np.ndarray]:
    keep = []
    for gene in range(data.shape[0]):
        if np.count_nonzero(data[gene,:]) >= min_cells:
            keep.append(gene)
    return data[np.asarray(keep),:],np.asarray(keep)

def library_size_normalize(data):
    m = np.median(np.sum(data,axis=0))
    data = data / np.sum(data,axis=0)
    data = data * m
    return data

def threshold_data(data,max_value=4.0):
    data[data> max_value] = max_value
    data[data< -max_value] = -max_value
    return data


In [ ]:
def get_cancer_spot(adata: sc.AnnData) -> np.ndarray:
    data = adata.X.toarray()
    data = data.T

    data,k = filter_genes(data,min_cells=int(data.shape[1]/20)) # 1
    data = library_size_normalize(data) #2
    data = np.log(data+1) 
    data = threshold_data(data,max_value=4.0)
    pca = PCA(n_components=1).fit_transform(data.T)
    km = KMeans(n_clusters=2).fit(pca)
    clusters = np.asarray(km.predict(pca))
    if np.mean(data[:,clusters==0]) < np.mean(data[:,clusters==1]):
        cancer_spots = np.asarray([x for x in range(data.shape[1])])[clusters==1]
    else:
        cancer_spots = np.asarray([x for x in range(data.shape[1])])[clusters==0]
    return cancer_spots

In [ ]:
def get_spot_assignment(adata, neighbors, name):
    label = adata.obs.loc[name,"CNV_assignment"]
    if label!="Tumor":
        return label
    
    vc = adata.obs.loc[neighbors[neighbors.loc[name]==1].index]["CNV_assignment"].value_counts()
    if vc.loc["Tumor"]<=int(vc.sum()/2)+1:
        return "Tumor_LE"
    else:
        return "Tumor_Core"
    

In [ ]:
def get_smoothed_label(label: str, vc: pd.DataFrame) -> str:
    if label=="Tumor":
        if vc.loc["Tumor"]<=1:
            return vc.idxmax()
        else:
            return label
    elif label=="Normal":
        if vc.loc["Normal"]<=1:
            return vc.idxmax()
        else:
            return label
    else:
        return label

def get_full_smoothed_assignment(adata: sc.AnnData) -> pd.DataFrame: 

    sq.gr.spatial_neighbors(adata, n_rings=1, coord_type="grid", n_neighs=6)
    
    neighbors = pd.DataFrame(adata.obsp["spatial_connectivities"].toarray(), 
                             index=adata.obs_names, columns=adata.obs_names)
    
    smoothed = []
    for name in adata.obs_names:
        label = adata.obs.loc[name,"CNV_assignment"]
        vc = adata.obs.loc[neighbors[neighbors.loc[name]==1].index]["CNV_assignment"].value_counts()
        if vc.sum()<=3:
            smoothed.append(label)
        else:
            smoothed.append(get_smoothed_label(label, vc))
    smoothed = pd.DataFrame(smoothed, index=adata.obs_names, columns=["Smoothed_assignment"])
    return smoothed

def update_closest(adata: sc.AnnData, n_rings: int, closest: pd.DataFrame, ass_col: str = "CNV_assignment") -> pd.DataFrame:
    sq.gr.spatial_neighbors(adata, n_rings=n_rings, coord_type="grid", n_neighs=6)

    neighbors = pd.DataFrame(adata.obsp["spatial_connectivities"].toarray(), 
                         index=adata.obs_names, columns=adata.obs_names)

    for name in adata.obs[adata.obs[ass_col]=="Tumor"].index:
        vc = adata.obs.loc[neighbors[neighbors.loc[name]==1].index][ass_col].value_counts()
        
        if vc.loc["Tumor"]<=vc.sum()-1:
            closest.loc[name,"Dist. to edge"] = min(n_rings,closest.loc[name,"Dist. to edge"])
    return closest

In [ ]:
from typing import Dict
def get_heatmap_interactions_tumor(sign_int: pd.DataFrame, 
                                   full_sigs: Dict[str, np.ndarray], 
                                   tumor_means: pd.DataFrame) -> pd.DataFrame:

    heatmap_df = {}

    for i,sig1 in enumerate(sorted(full_sigs)):
        df = sign_int.loc[sign_int.index.get_level_values(0).intersection(full_sigs[sig1])].reset_index()
        for j,sig2 in enumerate(sorted(full_sigs)):
            if j>=i:
                df1 = df[df.target.isin(np.intersect1d(df.target,full_sigs[sig2]))]
                if df1.shape[0]>0:
                    print(sig1,sig2)
                    pairs = [(df1.iloc[i].source,df1.iloc[i].target) for i in range(df1.shape[0])]
                    for pair in pairs:
                        if pair[0] in heatmap_df:
                            heatmap_df[pair[0]][pair[1]] = tumor_means.loc[pair]["Tumor_Core"]
                        else:
                            heatmap_df[pair[0]] = {pair[1]: tumor_means.loc[pair]["Tumor_Core"]}
    return pd.DataFrame(heatmap_df).T

def get_heatmap_interactions_tumor_to_normal(sign_int: pd.DataFrame, full_sigs: Dict[str, np.ndarray],
                                   normal_sigs: Dict[str, np.ndarray], tumor_means: pd.DataFrame) -> pd.DataFrame:

    heatmap_df = {}

    for i,sig1 in enumerate(sorted(full_sigs)):
        df = sign_int.loc[sign_int.index.get_level_values(0).intersection(full_sigs[sig1])].reset_index()
        for j,sig2 in enumerate(normal_sigs):
            if j>=i:
                df1 = df[df.target.isin(np.intersect1d(df.target,normal_sigs[sig2]))]
                if df1.shape[0]>0:
                    print(sig1,sig2)
                    pairs = [(df1.iloc[i].source,df1.iloc[i].target) for i in range(df1.shape[0])]
                    for pair in pairs:
                        if pair[0] in heatmap_df:
                            heatmap_df[pair[0]][pair[1]] = tumor_means.loc[pair]["Normal_Periphery"]
                        else:
                            heatmap_df[pair[0]] = {pair[1]: tumor_means.loc[pair]["Normal_Periphery"]}
    return pd.DataFrame(heatmap_df).T

def get_heatmap_interactions_normal_to_tumor(sign_int: pd.DataFrame, full_sigs: Dict[str, np.ndarray],
                                   normal_sigs: Dict[str, np.ndarray], normal_means: pd.DataFrame) -> pd.DataFrame:

    heatmap_df = {}

    for i,sig1 in enumerate(sorted(normal_sigs)):
        df = sign_int.loc[sign_int.index.get_level_values(0).intersection(normal_sigs[sig1])].reset_index()
        for j,sig2 in enumerate(full_sigs):
            if j>=i:
                df1 = df[df.target.isin(np.intersect1d(df.target,full_sigs[sig2]))]
                if df1.shape[0]>0:
                    print(sig1,sig2)
                    pairs = [(df1.iloc[i].source,df1.iloc[i].target) for i in range(df1.shape[0])]
                    for pair in pairs:
                        if pair[0] in heatmap_df:
                            heatmap_df[pair[0]][pair[1]] = normal_means.loc[pair]["Tumor_Periphery"]
                        else:
                            heatmap_df[pair[0]] = {pair[1]: normal_means.loc[pair]["Tumor_Periphery"]}
    return pd.DataFrame(heatmap_df).T

def return_color(list_idx: np.ndarray, 
                 sigs: Dict[str, np.ndarray], color_palette: Dict[str, str]) -> np.ndarray:
    color_mapping = {}
    for gene in list_idx:
        for sig, siggenes in sigs.items():
            if gene in siggenes:
                color_mapping[gene] = color_palette[sig]
    return color_mapping

def plot_interaction_heatmap(heatmap_df, color_mapping_idx, color_mapping_col, figsize=(2,1)) -> plt.Figure:
    fig, ax = plt.subplots(1,1,figsize=figsize)
    sns.heatmap(data=heatmap_df, cmap="vlag", center=0)
    ax.set_xticks(ax.get_xticks(), ax.get_xticklabels(), rotation=45, ha="right")
    for i,tick in enumerate(ax.get_xticklabels()):
        tick.set_color(color_mapping_col[tick.get_text()])
    for i,tick in enumerate(ax.get_yticklabels()):
        tick.set_color(color_mapping_idx[tick.get_text()])
    return fig

In [ ]:
def get_refined_location(adata: sc.AnnData, n_rings: int, location_refined: pd.DataFrame, 
                         ass_col: str = "CNV_assignment") -> pd.DataFrame:
    sq.gr.spatial_neighbors(adata, n_rings=n_rings, coord_type="grid", n_neighs=6)

    neighbors = pd.DataFrame(adata.obsp["spatial_connectivities"].toarray(), 
                         index=adata.obs_names, columns=adata.obs_names)

    for cell in neighbors.index:
    
        label = adata.obs.loc[cell,ass_col]

        vc = adata.obs.loc[neighbors[neighbors.loc[cell]==1].index][ass_col].value_counts()

        if label=="Tumor":
            if vc.loc["Tumor"]<=vc.sum()-1:
                location_refined.loc[cell,"Location refined"] = 'Tumor_Periphery'
            else:
                location_refined.loc[cell,"Location refined"] = 'Tumor_Core'
        elif label=="Normal":
            if vc.loc["Normal"]<=vc.sum()-1:
                location_refined.loc[cell,"Location refined"] = 'Normal_Periphery'
            else:
                location_refined.loc[cell,"Location refined"] = 'Normal_Healthy'
    return location_refined

In [ ]:
spatial_dir = pl.Path("/add/path/here/SpaceRanger_output/")

cell2location_results_dir = pl.Path("/add/path/here/Cell2Location_results/")

figure_dir = pl.Path("/add/path/here/")

visium_results_csv_dir = pl.Path("/add/path/here/visium_results_csv")

gencode_df = pd.read_csv("/add/path/here/gencode_v41_chr_mapping.csv",index_col=0)

# Download signatures

In [ ]:
signature_dir = pl.Path("/add/path/here/")

full_sigs = {}
for s in (signature_dir).iterdir():
    sig = s.stem
    full_sigs[sig] = pd.read_csv(s,index_col=0)
    full_sigs[sig] = full_sigs[sig].head(100).index.to_numpy()

In [ ]:
caf_dir = pl.Path("/add/path/here/")

caf_sigs = {}
for s in (caf_dir).iterdir():
    sig = s.stem
    caf_sigs[sig] = pd.read_csv(s,index_col=0)
    caf_sigs[sig] = caf_sigs[sig].head(100).names.to_numpy()

In [ ]:
myeloid_dir = pl.Path("/add/path/here/")

myeloid_sigs = {}
for s in (myeloid_dir).iterdir():
    sig = s.stem
    myeloid_sigs[sig] = pd.read_csv(s,index_col=0)
    myeloid_sigs[sig] = myeloid_sigs[sig].head(100).names.to_numpy()

In [ ]:
normal_sigs = (caf_sigs | myeloid_sigs)
del normal_sigs["HGF-CAF"]
del normal_sigs["Kupffer cells"]

# Download interactions for LIGREC

In [ ]:
interactions = pd.read_csv("/add/path/here/omnipath_intercell_network.csv",index_col=0)

df_interaction = interactions.loc[:,['genesymbol_intercell_source', 'genesymbol_intercell_target']]
df_interaction.columns = ["source","target"]
df_interaction.source = df_interaction.source.str.lstrip("COMPLEX:")
df_interaction.target = df_interaction.target.str.lstrip("COMPLEX:")

# EGSFR0074_A

In [ ]:
patient_name = "EGSFR0074_A"

In [ ]:
resdir = cell2location_results_dir / patient_name 

sample_path = resdir / patient_name

adata = sc.read_h5ad(resdir / "cell2location_map" / "sp.h5ad")

tissue_path = spatial_dir / patient_name / "spatial/tissue_positions_list.csv"
tissue_position = pd.read_csv(tissue_path,index_col=0)
tissue_position = tissue_position.loc[adata.obs_names]

#Set coordinates
x_array=tissue_position["array_row"].tolist()
y_array=tissue_position["array_col"].tolist()
x_pixel=tissue_position["pxl_row_in_fullres"].tolist()
y_pixel=tissue_position["pxl_col_in_fullres"].tolist()

x_min, x_max = np.min(x_pixel), np.max(x_pixel)
y_min, y_max = np.min(y_pixel), np.max(y_pixel)

cancer_spots = get_cancer_spot(adata=adata)

cancer_spot = pd.DataFrame(np.zeros(adata.shape[0]),index=adata.obs_names,columns=["Cancer spot"])
cancer_spot.loc[adata.obs_names[cancer_spots],"Cancer spot"] = 1

adata.obs["Cancer spot"] = cancer_spot.replace({0: "Normal", 1: "Tumor"})

In [ ]:
cell2loc_res = pd.read_csv(cell2location_results_dir / patient_name / "cell2location_map" / "celltype_abundance.csv",index_col=0)

# Get CNV profile

In [ ]:
sq.pl.spatial_scatter(adata, color=["Cancer spot"], crop_coord=(y_min, x_min, y_max, x_max), alpha=1)

In [ ]:
adata.var = pd.concat([adata.var,gencode_df.loc[adata.var_names.intersection(df.index)]],axis=1)

In [ ]:
cnv.tl.infercnv(
    adata,
    reference_key="Cancer spot",
    reference_cat=[
        "Normal",
    ],
    window_size=250,
)

cnv.tl.pca(adata)
cnv.pp.neighbors(adata)
cnv.tl.leiden(adata, resolution=0.3)

In [ ]:
cnv.pl.chromosome_heatmap(adata, groupby="cnv_leiden", dendrogram=True)

In [ ]:
cnv.tl.umap(adata)
cnv.tl.cnv_score(adata)

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(11, 11))
ax4.axis("off")
cnv.pl.umap(
    adata,
    color="cnv_leiden",
    legend_loc="on data",
    legend_fontoutline=2,
    ax=ax1,
    show=False,
)
cnv.pl.umap(adata, color="cnv_score", ax=ax2, show=False)
cnv.pl.umap(adata, color="Cancer spot", ax=ax3)

In [ ]:
adata.obs["CNV_assignment"] = adata.obs.cnv_leiden.replace({'2': "Tumor", '0': "Mixed", '1': "Normal"})

adata.obs[["Cancer spot","cnv_leiden","cnv_score","CNV_assignment"]].to_csv(f"{patient_name}_cnv_info.csv")

# Carcinoma component normalize

In [ ]:
adata.layers["counts"] = adata.X.copy()
adata.X = adata.layers["Carcinoma"]

sc.pp.normalize_total(adata, target_sum=10000)
sc.pp.log1p(adata)

gex = adata.to_df()
gex = (gex - gex.mean())/gex.std()

for sig, genes in full_sigs.items():
    adata.obs[sig] = gex[gex.columns.intersection(genes)].mean(axis=1)

In [ ]:
smoothed_labels = get_full_smoothed_assignment(adata)

adata.obs["Smoothed label"] = smoothed_labels
adata.obs["Smoothed label"] = adata.obs["Smoothed label"].astype("category")

ass_col = "Smoothed label"

closest = pd.DataFrame([1000]*adata.obs.shape[0],index=adata.obs.index,columns=["Dist. to edge"])

closest.loc[adata.obs[ass_col]=="Mixed"] = 0
closest.loc[adata.obs[ass_col]=="Normal"] = -1

for n_rings in np.arange(1,11):
    closest = update_closest(adata, n_rings=n_rings, closest=closest, ass_col=ass_col)

adata.obs["Dist. to edge"] = closest

order_filtered = sorted(closest.value_counts()[(closest.value_counts()>=5)].index.get_level_values(0))

fig, ax = plt.subplots(2,3,figsize=(10,5))
flatax = ax.flatten()

for i,prog in enumerate([f"cNMF_{i}" for i in range(1,6)]):
    sns.barplot(data=adata.obs, x="Dist. to edge", y=prog, ax=flatax[i], order=order_filtered)
    pretty_ax(flatax[i])
    labels = flatax[i].get_xticklabels()
    labels = ["Normal" if l.get_text()=="-1" else ("Mixed" if l.get_text()=="0" else f"d={l.get_text()}") for l in labels]
    flatax[i].set_xticks(flatax[i].get_xticks(), labels,rotation=45,ha="right")
    flatax[i].set_xlabel("")
    flatax[i].set_ylabel(f"cNMF$_{i+1}$")
flatax[-1].axis("off")
fig.tight_layout()
fig.savefig(figure_dir / patient_name / "cnmf_distance_to_center.svg", dpi=200, bbox_inches="tight")

In [ ]:
adata.obs[["Dist. to edge","cNMF_1","cNMF_2","cNMF_3","cNMF_4","cNMF_5"]].to_csv(visium_results_csv_dir / f"{patient_name}_cNMF_dist_to_edge.csv")
pd.Series(order_filtered).to_csv(visium_results_csv_dir / f"{patient_name}_dist_to_edge_order.csv")

In [ ]:
fig, ax = plt.subplots(2,3, figsize=(10,5))
flatax = ax.flatten()
sq.pl.spatial_scatter(adata, color=[f"cNMF_{i}" for i in range(1,6)]+["Smoothed label"], 
                      crop_coord=(y_min, x_min, y_max, x_max), 
                      alpha=1, ncols=3, 
                      vcenter=0, vmin=-0.3, 
                      vmax=0.3, fig=fig, ax=flatax)
for i in range(len(flatax)-1):
    flatax[i].set_title(f"cNMF$_{i+1}$")
flatax[-1].set_title("CNV-derived spot label")
fig.tight_layout()
fig.savefig(figure_dir / patient_name / "scatterplot_spatial.png", dpi=200, bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots(2,3,figsize=(7,3.5))
flatax = ax.flatten()

pairs = [("Normal","Tumor"),("Normal","Mixed"),("Mixed","Tumor")]
for i,prog in enumerate([f"cNMF_{i}" for i in range(1,6)]):
    sns.boxplot(data=adata.obs, x="Smoothed label", y=prog, ax=flatax[i], order=["Normal","Mixed","Tumor"])
    pretty_ax(flatax[i])
    flatax[i].set_xticks(flatax[i].get_xticks(),flatax[i].get_xticklabels(),rotation=45,ha="right")
    flatax[i].set_xlabel("")
    flatax[i].set_ylabel(f"cNMF$_{i+1}$")
    annotator = Annotator(flatax[i], pairs, data=adata.obs, x="Smoothed label", 
                          y=prog, order=["Normal","Mixed","Tumor"])
    annotator.configure(test='Mann-Whitney', text_format='simple', show_test_name=False, loc='inside')
    annotator.apply_and_annotate()
flatax[-1].axis("off")
fig.tight_layout()
fig.savefig(figure_dir / patient_name / "dist_cnmf_cnv_assignemnt.png", dpi=200, bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots(2,3,figsize=(7,3.5))
flatax = ax.flatten()
list_ct = ["Carcinoma","Fibroblast","Endothelial","Myeloid","Lymphoid","Muscle"]
for i in range(len(list_ct)):
    sns.boxplot(data=adata.obs, x="Smoothed label", y=list_ct[i], ax=flatax[i], order=["Normal","Mixed","Tumor"])
    pretty_ax(flatax[i])
    flatax[i].set_ylabel("N cells")
    flatax[i].set_xlabel("")
    flatax[i].set_title(f"{list_ct[i]}")
    flatax[i].set_xticks(flatax[i].get_xticks(), flatax[i].get_xticklabels(), rotation=45, ha="right")
    
fig.tight_layout()
fig.savefig(figure_dir / patient_name / "N_cells_perspot.png", dpi=200, bbox_inches="tight")

In [ ]:
location_refined = adata.obs[[ass_col]].copy().astype(str)
location_refined.columns = ["Location refined"]

adata.obs["Location refined"] = get_refined_location(adata=adata, n_rings=1, 
                                                     location_refined=location_refined, ass_col="Smoothed label")
adata.obs["Location refined"] = adata.obs["Location refined"].astype("category")

# Run LIGREC

In [ ]:
adata.X = adata.layers["counts"].copy()
sc.pp.normalize_total(adata, target_sum=10000)

In [ ]:
sq.gr.ligrec(
    adata, use_raw=False, interactions=df_interaction,
    n_perms=500, corr_method="fdr_bh",
    cluster_key="Location refined",
)

tumor_interactions = adata.uns["Location refined_ligrec"]["pvalues"]["Tumor_Periphery"]

tumor_means = adata.uns["Location refined_ligrec"]["means"]["Tumor_Periphery"]

normal_interactions = adata.uns["Location refined_ligrec"]["pvalues"]["Normal_Periphery"]

normal_means = adata.uns["Location refined_ligrec"]["means"]["Normal_Periphery"]

mixed_interactions = adata.uns["Location refined_ligrec"]["pvalues"]["Mixed"]

mixed_means = adata.uns["Location refined_ligrec"]["means"]["Mixed"]

In [ ]:
sign_int = tumor_interactions["Tumor_Periphery"][tumor_interactions["Tumor_Periphery"]<0.05]

heatmap_tumor_df = get_heatmap_interactions_tumor(sign_int=sign_int, full_sigs=full_sigs, tumor_means=tumor_means)

sign_int = tumor_interactions["Normal_Periphery"][tumor_interactions["Normal_Periphery"]<0.05]

heatmap_tumor_to_normal_df = get_heatmap_interactions_tumor_to_normal(sign_int=sign_int, full_sigs=full_sigs, 
                                                             normal_sigs=normal_sigs, tumor_means=tumor_means)

sign_int = normal_interactions["Tumor_Periphery"][normal_interactions["Tumor_Periphery"]<0.05]

heatmap_normal_to_tumor_df = get_heatmap_interactions_normal_to_tumor(sign_int=sign_int, full_sigs=full_sigs, 
                                                             normal_sigs=normal_sigs, normal_means=normal_means)

In [ ]:
import palettable
colorlist = palettable.colorbrewer.qualitative.Set1_7.mpl_colors
colormapping_mal = {"cNMF_1": colorlist[0], "cNMF_2": colorlist[1], "cNMF_3": colorlist[3], 
                    "cNMF_4": colorlist[4], "cNMF_5": colorlist[6]}

colorlist = palettable.colorbrewer.sequential.Greens_9.mpl_colors
colormapping_myeloid = {"TAM1": colorlist[1], "DC": colorlist[2], 
                        "Mast": colorlist[4], "TAM2": colorlist[5]}

colorlist = palettable.colorbrewer.sequential.Oranges_5.mpl_colors
colormapping_fibroblast = {"Inflammatory CAF": colorlist[1], "Adipose CAF": colorlist[3], 
                           "Fibroblast": colorlist[4]}

colormapping_normal = (colormapping_myeloid | colormapping_fibroblast)

In [ ]:
color_mapping_idx = return_color(list_idx = heatmap_normal_to_tumor_df.index.to_numpy(), 
                             sigs = normal_sigs, color_palette = colormapping_normal)
color_mapping_col = return_color(list_idx = heatmap_normal_to_tumor_df.columns.to_numpy(), 
                             sigs = full_sigs, color_palette = colormapping_mal)

fig = plot_interaction_heatmap(heatmap_normal_to_tumor_df, color_mapping_idx, color_mapping_col, figsize=(2,1.5))
fig.savefig(figure_dir / patient_name / "normal_to_tumor_LIGREC.svg", dpi=200, bbox_inches="tight")

In [ ]:
adata.uns["Location refined_ligrec"]["means"].to_csv(visium_results_csv_dir / f"{patient_name}_LIGREC_means.csv")
adata.uns["Location refined_ligrec"]["pvalues"].to_csv(visium_results_csv_dir / f"{patient_name}_LIGREC_pvalues.csv")

# EGSFR1938_A

In [ ]:
patient_name = "EGSFR1938_A"

In [ ]:
resdir = cell2location_results_dir / patient_name 

sample_path = resdir / patient_name

adata = sc.read_h5ad(resdir / "cell2location_map" / "sp.h5ad")

tissue_path = spatial_dir / patient_name / "spatial/tissue_positions_list.csv"
tissue_position = pd.read_csv(tissue_path,index_col=0)
tissue_position = tissue_position.loc[adata.obs_names]

#Set coordinates
x_array=tissue_position["array_row"].tolist()
y_array=tissue_position["array_col"].tolist()
x_pixel=tissue_position["pxl_row_in_fullres"].tolist()
y_pixel=tissue_position["pxl_col_in_fullres"].tolist()

x_min, x_max = np.min(x_pixel), np.max(x_pixel)
y_min, y_max = np.min(y_pixel), np.max(y_pixel)

cancer_spots = get_cancer_spot(adata=adata)

cancer_spot = pd.DataFrame(np.zeros(adata.shape[0]),index=adata.obs_names,columns=["Cancer spot"])
cancer_spot.loc[adata.obs_names[cancer_spots],"Cancer spot"] = 1

adata.obs["Cancer spot"] = cancer_spot.replace({0: "Normal", 1: "Tumor"})

In [ ]:
cell2loc_res = pd.read_csv(cell2location_results_dir / patient_name / "cell2location_map" / "celltype_abundance.csv",index_col=0)

# Get CNV profile

In [ ]:
sq.pl.spatial_scatter(adata, color=["Cancer spot"], crop_coord=(y_min, x_min, y_max, x_max), alpha=1)

In [ ]:
adata.var = pd.concat([adata.var,gencode_df.loc[adata.var_names.intersection(df.index)]],axis=1)

In [ ]:
cnv.tl.infercnv(
    adata,
    reference_key="Cancer spot",
    reference_cat=[
        "Normal",
    ],
    window_size=250,
)

cnv.tl.pca(adata)
cnv.pp.neighbors(adata)
cnv.tl.leiden(adata, resolution=0.2)

In [ ]:
cnv.pl.chromosome_heatmap(adata, groupby="cnv_leiden", dendrogram=True)

In [ ]:
cnv.tl.umap(adata)
cnv.tl.cnv_score(adata)

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(11, 11))
ax4.axis("off")
cnv.pl.umap(
    adata,
    color="cnv_leiden",
    legend_loc="on data",
    legend_fontoutline=2,
    ax=ax1,
    show=False,
)
cnv.pl.umap(adata, color="cnv_score", ax=ax2, show=False)
cnv.pl.umap(adata, color="Cancer spot", ax=ax3)

In [ ]:
adata.obs["CNV_assignment"] = adata.obs.cnv_leiden.replace({'3': "Tumor", '0': "Normal", '1': "Normal", 
                                                            '2': "Normal", '4': "Normal", })#"5": "Normal"

adata.obs[["Cancer spot","cnv_leiden","cnv_score","CNV_assignment"]].to_csv(f"{patient_name}_cnv_info.csv")

# Carcinoma component normalize

In [ ]:
adata.layers["counts"] = adata.X.copy()
adata.X = adata.layers["Carcinoma"]

sc.pp.normalize_total(adata, target_sum=10000)
sc.pp.log1p(adata)

gex = adata.to_df()
gex = (gex - gex.mean())/gex.std()

for sig, genes in full_sigs.items():
    adata.obs[sig] = gex[gex.columns.intersection(genes)].mean(axis=1)

In [ ]:
adata.X = adata.layers["Fibroblast"]

sc.pp.normalize_total(adata, target_sum=10000)
sc.pp.log1p(adata)

gex = adata.to_df()
gex = (gex - gex.mean())/gex.std()

for sig, genes in caf_sigs.items():
    adata.obs[sig] = gex[gex.columns.intersection(genes)].mean(axis=1)

In [ ]:
adata.X = adata.layers["Myeloid"]

sc.pp.normalize_total(adata, target_sum=10000)
sc.pp.log1p(adata)

gex = adata.to_df()
gex = (gex - gex.mean())/gex.std()

for sig, genes in myeloid_sigs.items():
    adata.obs[sig] = gex[gex.columns.intersection(genes)].mean(axis=1)

In [ ]:
smoothed_labels = get_full_smoothed_assignment(adata)

adata.obs["Smoothed label"] = smoothed_labels
adata.obs["Smoothed label"] = adata.obs["Smoothed label"].astype("category")

ass_col = "Smoothed label"

closest = pd.DataFrame([1000]*adata.obs.shape[0],index=adata.obs.index,columns=["Dist. to edge"])

closest.loc[adata.obs[ass_col]=="Mixed"] = 0
closest.loc[adata.obs[ass_col]=="Normal"] = -1

for n_rings in np.arange(1,11):
    closest = update_closest(adata, n_rings=n_rings, closest=closest, ass_col=ass_col)

adata.obs["Dist. to edge"] = closest

order_filtered = sorted(closest.value_counts()[(closest.value_counts()>=5)].index.get_level_values(0))

fig, ax = plt.subplots(2,3,figsize=(10,5))
flatax = ax.flatten()

for i,prog in enumerate([f"cNMF_{i}" for i in range(1,6)]):
    sns.barplot(data=adata.obs, x="Dist. to edge", y=prog, ax=flatax[i], order=order_filtered)
    pretty_ax(flatax[i])
    labels = flatax[i].get_xticklabels()
    labels = ["Normal" if l.get_text()=="-1" else ("Mixed" if l.get_text()=="0" else f"d={l.get_text()}") for l in labels]
    flatax[i].set_xticks(flatax[i].get_xticks(), labels,rotation=45,ha="right")
    flatax[i].set_xlabel("")
    flatax[i].set_ylabel(f"cNMF$_{i+1}$")
flatax[-1].axis("off")
fig.tight_layout()
fig.savefig(figure_dir / patient_name / "cnmf_distance_to_center.svg", dpi=200, bbox_inches="tight")

In [ ]:
adata.obs[["Dist. to edge","cNMF_1","cNMF_2","cNMF_3","cNMF_4","cNMF_5"]].to_csv(visium_results_csv_dir / f"{patient_name}_cNMF_dist_to_edge.csv")
pd.Series(order_filtered).to_csv(visium_results_csv_dir / f"{patient_name}_dist_to_edge_order.csv")

In [ ]:
fig, ax = plt.subplots(2,3, figsize=(10,5))
flatax = ax.flatten()
sq.pl.spatial_scatter(adata, color=[f"cNMF_{i}" for i in range(1,6)]+["Smoothed label"], 
                      crop_coord=(y_min, x_min, y_max, x_max), 
                      alpha=1, ncols=3, 
                      vcenter=0, vmin=-0.4, 
                      vmax=0.4, fig=fig, ax=flatax)
for i in range(len(flatax)-1):
    flatax[i].set_title(f"cNMF$_{i+1}$")
flatax[-1].set_title("CNV-derived spot label")
fig.tight_layout()
fig.savefig(figure_dir / patient_name / "scatterplot_spatial.png", dpi=200, bbox_inches="tight")
fig.savefig(figure_dir / patient_name / "scatterplot_spatial.svg", dpi=100, bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots(1,4, figsize=(12,2.6))
flatax = ax.flatten()
sq.pl.spatial_scatter(adata, color=["Inflammatory CAF","Adipose CAF","Fibroblast"]+["Smoothed label"], 
                      crop_coord=(y_min, x_min, y_max, x_max), 
                      alpha=1, ncols=3, 
                      vcenter=0, vmin=-0.4, 
                      vmax=0.4, fig=fig, ax=flatax)

flatax[-1].set_title("CNV-derived spot label")
fig.tight_layout()
fig.savefig(figure_dir / patient_name / "scatterplot_fibro.png", dpi=200, bbox_inches="tight")
fig.savefig(figure_dir / patient_name / "scatterplot_fibro.svg", dpi=100, bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots(2,3, figsize=(10,5))
flatax = ax.flatten()
sq.pl.spatial_scatter(adata, color=list(myeloid_sigs.keys())+["Smoothed label"], 
                      crop_coord=(y_min, x_min, y_max, x_max), 
                      alpha=1, ncols=3, 
                      vcenter=0, vmin=-0.4, 
                      vmax=0.4, fig=fig, ax=flatax)

flatax[-1].set_title("CNV-derived spot label")
fig.tight_layout()
fig.savefig(figure_dir / patient_name / "scatterplot_myeloid.png", dpi=200, bbox_inches="tight")
fig.savefig(figure_dir / patient_name / "scatterplot_myeloid.svg", dpi=100, bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots(2,3,figsize=(7,3.5))
flatax = ax.flatten()
list_ct = ["Carcinoma","Fibroblast","Endothelial","Myeloid","Lymphoid","Muscle"]
for i in range(len(list_ct)):
    sns.boxplot(data=adata.obs, x="Smoothed label", y=list_ct[i], ax=flatax[i], order=["Normal","Tumor"])
    pretty_ax(flatax[i])
    flatax[i].set_ylabel("N cells")
    flatax[i].set_xlabel("")
    flatax[i].set_title(f"{list_ct[i]}")
    flatax[i].set_xticks(flatax[i].get_xticks(), flatax[i].get_xticklabels(), rotation=45, ha="right")
    
fig.tight_layout()
fig.savefig(figure_dir / patient_name / "N_cells_perspot.png", dpi=200, bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots(2,3,figsize=(7,3.5))
flatax = ax.flatten()

pairs = [("Normal","Tumor")]
for i,prog in enumerate([f"cNMF_{i}" for i in range(1,6)]):
    sns.boxplot(data=adata.obs, x="Smoothed label", y=prog, ax=flatax[i], order=["Normal","Tumor"])
    pretty_ax(flatax[i])
    flatax[i].set_xticks(flatax[i].get_xticks(),flatax[i].get_xticklabels(),rotation=45,ha="right")
    flatax[i].set_xlabel("")
    flatax[i].set_ylabel(f"cNMF$_{i+1}$")
    annotator = Annotator(flatax[i], pairs, data=adata.obs, x="Smoothed label", 
                          y=prog, order=["Normal","Tumor"])
    annotator.configure(test='Mann-Whitney', text_format='simple', show_test_name=False, loc='inside')
    annotator.apply_and_annotate()
flatax[-1].axis("off")
fig.tight_layout()
fig.savefig(figure_dir / patient_name / "dist_cnmf_cnv_assignemnt.png", dpi=200, bbox_inches="tight")

In [ ]:
location_refined = adata.obs[[ass_col]].copy().astype(str)
location_refined.columns = ["Location refined"]

adata.obs["Location refined"] = get_refined_location(adata=adata, n_rings=1, 
                                                     location_refined=location_refined, ass_col="Smoothed label")
adata.obs["Location refined"] = adata.obs["Location refined"].astype("category")

# Run LIGREC

In [ ]:
adata.X = adata.layers["counts"].copy()
sc.pp.normalize_total(adata, target_sum=10000)

In [ ]:
sq.gr.ligrec(
    adata, use_raw=False, interactions=df_interaction,
    n_perms=500, corr_method="fdr_bh",
    cluster_key="Location refined",
)

tumor_per_interactions = adata.uns["Location refined_ligrec"]["pvalues"]["Tumor_Periphery"]

tumor_per_means = adata.uns["Location refined_ligrec"]["means"]["Tumor_Periphery"]

tumor_core_interactions = adata.uns["Location refined_ligrec"]["pvalues"]["Tumor_Core"]

tumor_core_means = adata.uns["Location refined_ligrec"]["means"]["Tumor_Core"]

normal_interactions = adata.uns["Location refined_ligrec"]["pvalues"]["Normal_Periphery"]

normal_means = adata.uns["Location refined_ligrec"]["means"]["Normal_Periphery"]

In [ ]:
sign_int = tumor_core_interactions["Tumor_Core"][tumor_core_interactions["Tumor_Core"]<0.05]

heatmap_tumor_df = get_heatmap_interactions_tumor(sign_int=sign_int, full_sigs=full_sigs, tumor_means=tumor_core_means)

sign_int = tumor_per_interactions["Normal_Periphery"][tumor_per_interactions["Normal_Periphery"]<0.05]

heatmap_tumor_to_normal_df = get_heatmap_interactions_tumor_to_normal(sign_int=sign_int, full_sigs=full_sigs, 
                                                             normal_sigs=normal_sigs, tumor_means=tumor_per_means)

sign_int = normal_interactions["Tumor_Periphery"][normal_interactions["Tumor_Periphery"]<0.05]

heatmap_normal_to_tumor_df = get_heatmap_interactions_normal_to_tumor(sign_int=sign_int, full_sigs=full_sigs, 
                                                             normal_sigs=normal_sigs, normal_means=normal_means)

In [ ]:
import palettable
colorlist = palettable.colorbrewer.qualitative.Set1_7.mpl_colors
colormapping_mal = {"cNMF_1": colorlist[0], "cNMF_2": colorlist[1], "cNMF_3": colorlist[3], 
                    "cNMF_4": colorlist[4], "cNMF_5": colorlist[6]}

colorlist = palettable.colorbrewer.sequential.Greens_9.mpl_colors
colormapping_myeloid = {"TAM1": colorlist[1], "DC": colorlist[2], 
                        "Mast": colorlist[4], "TAM2": colorlist[5]}

colorlist = palettable.colorbrewer.sequential.Oranges_5.mpl_colors
colormapping_fibroblast = {"Inflammatory CAF": colorlist[1], "Adipose CAF": colorlist[3], 
                           "Fibroblast": colorlist[4]}

colormapping_normal = (colormapping_myeloid | colormapping_fibroblast)

In [ ]:
color_mapping_idx = return_color(list_idx = heatmap_tumor_to_normal_df.index.to_numpy(), 
                             sigs = full_sigs, color_palette = colormapping_mal)
color_mapping_col = return_color(list_idx = heatmap_tumor_to_normal_df.columns.to_numpy(), 
                             sigs = normal_sigs, color_palette = colormapping_normal)

fig = plot_interaction_heatmap(heatmap_tumor_to_normal_df, color_mapping_idx, color_mapping_col, figsize=(2,1.5))
fig.savefig(figure_dir / patient_name / "tumor_to_normal_LIGREC.svg", dpi=200, bbox_inches="tight")

In [ ]:
color_mapping_idx = return_color(list_idx = heatmap_normal_to_tumor_df.index.to_numpy(), 
                             sigs = normal_sigs, color_palette = colormapping_normal)
color_mapping_col = return_color(list_idx = heatmap_normal_to_tumor_df.columns.to_numpy(), 
                             sigs = full_sigs, color_palette = colormapping_mal)

fig = plot_interaction_heatmap(heatmap_normal_to_tumor_df, color_mapping_idx, color_mapping_col, figsize=(2,1.5))
fig.savefig(figure_dir / patient_name / "normal_to_tumor_LIGREC.svg", dpi=200, bbox_inches="tight")

In [ ]:
adata.uns["Location refined_ligrec"]["means"].to_csv(visium_results_csv_dir / f"{patient_name}_LIGREC_means.csv")
adata.uns["Location refined_ligrec"]["pvalues"].to_csv(visium_results_csv_dir / f"{patient_name}_LIGREC_pvalues.csv")

In [ ]:
# want to show in log scale
sc.pp.log1p(adata)

In [ ]:
LR_list = ["TGFBI","ITGB1","FN1","ITGA4","FN1","ITGA2","APP","TNFRSF21",]
LR_color_list = [colormapping_mal["cNMF_3"],colormapping_fibroblast["Adipose CAF"],
                colormapping_mal["cNMF_3"],colormapping_myeloid["DC"],
                colormapping_fibroblast["Adipose CAF"],colormapping_mal["cNMF_4"],
                colormapping_myeloid["TAM2"],colormapping_mal["cNMF_4"],]

fig, ax = plt.subplots(4,2, figsize=(7,8))
flatax = ax.flatten()
sq.pl.spatial_scatter(adata, color=LR_list, 
                      crop_coord=(y_min, x_min, y_max, x_max), 
                      alpha=1, ncols=2, 
                      fig=fig, ax=flatax)
for i in range(len(flatax)):
    flatax[i].set_title(LR_list[i], c=LR_color_list[i])
fig.tight_layout()
fig.savefig(figure_dir / patient_name / "spatial_rep_LR_LIGREC_full.svg", dpi=200, bbox_inches="tight")

In [ ]:
LR_list = ["FN1","VCAN","BIRC5","ITGA2"]
LR_color_list = [colormapping_fibroblast["Adipose CAF"],colormapping_mal["cNMF_3"],
                colormapping_mal["cNMF_2"],
                colormapping_mal["cNMF_4"],]

fig, ax = plt.subplots(1,4, figsize=(7,7))
flatax = ax.flatten()
sq.pl.spatial_scatter(adata, color=LR_list, 
                      crop_coord=(y_min, x_min, y_max, x_max), 
                      alpha=1, ncols=4, fig=fig, ax=flatax)
for i in range(len(flatax)):
    flatax[i].set_title(LR_list[i], c=LR_color_list[i])
fig.tight_layout()
fig.savefig(figure_dir / patient_name / "spatial_rep_LR_LIGREC.svg", dpi=200, bbox_inches="tight")

# EGSFR0148

In [ ]:
patient_name = "EGSFR0148"

In [ ]:
resdir = cell2location_results_dir / patient_name 

sample_path = resdir / patient_name

adata = sc.read_h5ad(resdir / "cell2location_map" / "sp.h5ad")

tissue_path = spatial_dir / patient_name / "spatial/tissue_positions_list.csv"
tissue_position = pd.read_csv(tissue_path,index_col=0)
tissue_position = tissue_position.loc[adata.obs_names]

#Set coordinates
x_array=tissue_position["array_row"].tolist()
y_array=tissue_position["array_col"].tolist()
x_pixel=tissue_position["pxl_row_in_fullres"].tolist()
y_pixel=tissue_position["pxl_col_in_fullres"].tolist()

x_min, x_max = np.min(x_pixel), np.max(x_pixel)
y_min, y_max = np.min(y_pixel), np.max(y_pixel)

cancer_spots = get_cancer_spot(adata=adata)

cancer_spot = pd.DataFrame(np.zeros(adata.shape[0]),index=adata.obs_names,columns=["Cancer spot"])
cancer_spot.loc[adata.obs_names[cancer_spots],"Cancer spot"] = 1

adata.obs["Cancer spot"] = cancer_spot.replace({0: "Normal", 1: "Tumor"})

# Get carcinoma-specific expression and save in layers

In [ ]:
cell2loc_res = pd.read_csv(cell2location_results_dir / patient_name / "cell2location_map" / "celltype_abundance.csv",index_col=0)

# Get CNV profile

In [ ]:
adata.var = pd.concat([adata.var,gencode_df.loc[adata.var_names.intersection(df.index)]],axis=1)

In [ ]:
cnv.tl.infercnv(
    adata,
    reference_key="Cancer spot",
    reference_cat=[
        "Normal",
    ],
    window_size=250,
)

cnv.tl.pca(adata)
cnv.pp.neighbors(adata)
cnv.tl.leiden(adata, resolution=0.3)

In [ ]:
cnv.pl.chromosome_heatmap(adata, groupby="cnv_leiden", dendrogram=True)

In [ ]:
cnv.tl.umap(adata)
cnv.tl.cnv_score(adata)

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(11, 11))
ax4.axis("off")
cnv.pl.umap(
    adata,
    color="cnv_leiden",
    legend_loc="on data",
    legend_fontoutline=2,
    ax=ax1,
    show=False,
)
cnv.pl.umap(adata, color="cnv_score", ax=ax2, show=False)
cnv.pl.umap(adata, color="Cancer spot", ax=ax3)

In [ ]:
adata.obs["CNV_assignment"] = adata.obs.cnv_leiden.replace({'0': "Normal", '1': "Tumor", 
                                                            '2': "Normal", '3': "Mixed"})

adata.obs[["Cancer spot","cnv_leiden","cnv_score","CNV_assignment"]].to_csv(f"{patient_name}_cnv_info.csv")

# Carcinoma component normalize

In [ ]:
adata.layers["counts"] = adata.X.copy()
adata.X = adata.layers["Carcinoma"]

sc.pp.normalize_total(adata, target_sum=10000)
sc.pp.log1p(adata)

gex = adata.to_df()
gex = (gex - gex.mean())/gex.std()

for sig, genes in full_sigs.items():
    adata.obs[sig] = gex[gex.columns.intersection(genes)].mean(axis=1)

In [ ]:
adata.X = adata.layers["Fibroblast"]

sc.pp.normalize_total(adata, target_sum=10000)
sc.pp.log1p(adata)

gex = adata.to_df()
gex = (gex - gex.mean())/gex.std()

for sig, genes in caf_sigs.items():
    adata.obs[sig] = gex[gex.columns.intersection(genes)].mean(axis=1)

In [ ]:
adata.X = adata.layers["Myeloid"]

sc.pp.normalize_total(adata, target_sum=10000)
sc.pp.log1p(adata)

gex = adata.to_df()
gex = (gex - gex.mean())/gex.std()

for sig, genes in myeloid_sigs.items():
    adata.obs[sig] = gex[gex.columns.intersection(genes)].mean(axis=1)

In [ ]:
smoothed_labels = get_full_smoothed_assignment(adata)

adata.obs["Smoothed label"] = smoothed_labels
adata.obs["Smoothed label"] = adata.obs["Smoothed label"].astype("category")

ass_col = "Smoothed label"

closest = pd.DataFrame([1000]*adata.obs.shape[0],index=adata.obs.index,columns=["Dist. to edge"])

closest.loc[adata.obs[ass_col]=="Mixed"] = 0
closest.loc[adata.obs[ass_col]=="Normal"] = -1

for n_rings in np.arange(1,11):
    closest = update_closest(adata, n_rings=n_rings, closest=closest, ass_col=ass_col)

adata.obs["Dist. to edge"] = closest

order_filtered = sorted(closest.value_counts()[(closest.value_counts()>=5)].index.get_level_values(0))

fig, ax = plt.subplots(2,3,figsize=(10,5))
flatax = ax.flatten()

for i,prog in enumerate([f"cNMF_{i}" for i in range(1,6)]):
    sns.barplot(data=adata.obs, x="Dist. to edge", y=prog, ax=flatax[i], order=order_filtered)
    pretty_ax(flatax[i])
    labels = flatax[i].get_xticklabels()
    labels = ["Normal" if l.get_text()=="-1" else ("Mixed" if l.get_text()=="0" else f"d={l.get_text()}") for l in labels]
    flatax[i].set_xticks(flatax[i].get_xticks(), labels,rotation=45,ha="right")
    flatax[i].set_xlabel("")
    flatax[i].set_ylabel(f"cNMF$_{i+1}$")
flatax[-1].axis("off")
fig.tight_layout()
fig.savefig(figure_dir / patient_name / "cnmf_distance_to_center.svg", dpi=200, bbox_inches="tight")

In [ ]:
adata.obs[["Dist. to edge","cNMF_1","cNMF_2","cNMF_3","cNMF_4","cNMF_5"]].to_csv(visium_results_csv_dir / f"{patient_name}_cNMF_dist_to_edge.csv")
pd.Series(order_filtered).to_csv(visium_results_csv_dir / f"{patient_name}_dist_to_edge_order.csv")

In [ ]:
fig, ax = plt.subplots(2,3, figsize=(10,5))
flatax = ax.flatten()
sq.pl.spatial_scatter(adata, color=[f"cNMF_{i}" for i in range(1,6)]+["Smoothed label"], 
                      crop_coord=(y_min, x_min, y_max, x_max), 
                      alpha=1, ncols=3, 
                      vcenter=0, vmin=-0.3, 
                      vmax=0.3, fig=fig, ax=flatax)
for i in range(len(flatax)-1):
    flatax[i].set_title(f"cNMF$_{i+1}$")
flatax[-1].set_title("CNV-derived spot label")
fig.tight_layout()
fig.savefig(figure_dir / patient_name / "scatterplot_spatial.png", dpi=200, bbox_inches="tight")
fig.savefig(figure_dir / patient_name / "scatterplot_spatial.svg", dpi=100, bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots(1,4, figsize=(13,2.5))
flatax = ax.flatten()
sq.pl.spatial_scatter(adata, color=["Inflammatory CAF","Adipose CAF","Fibroblast"]+["Smoothed label"], 
                      crop_coord=(y_min, x_min, y_max, x_max), 
                      alpha=1, ncols=3, 
                      vcenter=0, vmin=-0.3, 
                      vmax=0.3, fig=fig, ax=flatax)

flatax[-1].set_title("CNV-derived spot label")
fig.tight_layout()
fig.savefig(figure_dir / patient_name / "scatterplot_fibro.png", dpi=200, bbox_inches="tight")
fig.savefig(figure_dir / patient_name / "scatterplot_fibro.svg", dpi=100, bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots(2,3, figsize=(10,5))
flatax = ax.flatten()
sq.pl.spatial_scatter(adata, color=list(myeloid_sigs.keys())+["Smoothed label"], 
                      crop_coord=(y_min, x_min, y_max, x_max), 
                      alpha=1, ncols=3, 
                      vcenter=0, vmin=-0.3, 
                      vmax=0.3, fig=fig, ax=flatax)

flatax[-1].set_title("CNV-derived spot label")
fig.tight_layout()
fig.savefig(figure_dir / patient_name / "scatterplot_myeloid.png", dpi=200, bbox_inches="tight")
fig.savefig(figure_dir / patient_name / "scatterplot_myeloid.svg", dpi=100, bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots(2,3,figsize=(7,3.5))
flatax = ax.flatten()
list_ct = ["Carcinoma","Fibroblast","Endothelial","Myeloid","Lymphoid","Muscle"]
for i in range(len(list_ct)):
    sns.boxplot(data=adata.obs, x="Smoothed label", y=list_ct[i], ax=flatax[i], order=["Normal","Mixed","Tumor"])
    pretty_ax(flatax[i])
    flatax[i].set_ylabel("N cells")
    flatax[i].set_xlabel("")
    flatax[i].set_title(f"{list_ct[i]}")
    flatax[i].set_xticks(flatax[i].get_xticks(), flatax[i].get_xticklabels(), rotation=45, ha="right")
    
fig.tight_layout()
fig.savefig(figure_dir / patient_name / "N_cells_perspot.png", dpi=200, bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots(2,3,figsize=(7,3.5))
flatax = ax.flatten()

pairs = [("Normal","Tumor"),("Normal","Mixed"),("Mixed","Tumor")]
for i,prog in enumerate([f"cNMF_{i}" for i in range(1,6)]):
    sns.boxplot(data=adata.obs, x="Smoothed label", y=prog, ax=flatax[i], order=["Normal","Mixed","Tumor"])
    pretty_ax(flatax[i])
    flatax[i].set_xticks(flatax[i].get_xticks(),flatax[i].get_xticklabels(),rotation=45,ha="right")
    flatax[i].set_xlabel("")
    flatax[i].set_ylabel(f"cNMF$_{i+1}$")
    annotator = Annotator(flatax[i], pairs, data=adata.obs, x="Smoothed label", 
                          y=prog, order=["Normal","Mixed","Tumor"])
    annotator.configure(test='Mann-Whitney', text_format='simple', show_test_name=False, loc='inside')
    annotator.apply_and_annotate()
flatax[-1].axis("off")
fig.tight_layout()
fig.savefig(figure_dir / patient_name / "dist_cnmf_cnv_assignemnt.png", dpi=200, bbox_inches="tight")

In [ ]:
location_refined = adata.obs[[ass_col]].copy().astype(str)
location_refined.columns = ["Location refined"]

adata.obs["Location refined"] = get_refined_location(adata=adata, n_rings=1, 
                                                     location_refined=location_refined, ass_col="Smoothed label")
adata.obs["Location refined"] = adata.obs["Location refined"].astype("category")

# Run LIGREC

In [ ]:
adata.X = adata.layers["counts"].copy()
sc.pp.normalize_total(adata, target_sum=10000)

In [ ]:
sq.gr.ligrec(
    adata, use_raw=False, interactions=df_interaction,
    n_perms=500, corr_method="fdr_bh",
    cluster_key="Location refined",
)

tumor_per_interactions = adata.uns["Location refined_ligrec"]["pvalues"]["Tumor_Periphery"]

tumor_per_means = adata.uns["Location refined_ligrec"]["means"]["Tumor_Periphery"]

tumor_core_interactions = adata.uns["Location refined_ligrec"]["pvalues"]["Tumor_Core"]

tumor_core_means = adata.uns["Location refined_ligrec"]["means"]["Tumor_Core"]

normal_interactions = adata.uns["Location refined_ligrec"]["pvalues"]["Normal_Periphery"]

normal_means = adata.uns["Location refined_ligrec"]["means"]["Normal_Periphery"]

In [ ]:
sign_int = tumor_core_interactions["Tumor_Core"][tumor_core_interactions["Tumor_Core"]<0.05]

heatmap_tumor_df = get_heatmap_interactions_tumor(sign_int=sign_int, full_sigs=full_sigs, tumor_means=tumor_core_means)

sign_int = tumor_per_interactions["Normal_Periphery"][tumor_per_interactions["Normal_Periphery"]<0.05]

heatmap_tumor_to_normal_df = get_heatmap_interactions_tumor_to_normal(sign_int=sign_int, full_sigs=full_sigs, 
                                                             normal_sigs=normal_sigs, tumor_means=tumor_per_means)

sign_int = normal_interactions["Tumor_Periphery"][normal_interactions["Tumor_Periphery"]<0.05]

heatmap_normal_to_tumor_df = get_heatmap_interactions_normal_to_tumor(sign_int=sign_int, full_sigs=full_sigs, 
                                                             normal_sigs=normal_sigs, normal_means=normal_means)

In [ ]:
import palettable
colorlist = palettable.colorbrewer.qualitative.Set1_7.mpl_colors
colormapping_mal = {"cNMF_1": colorlist[0], "cNMF_2": colorlist[1], "cNMF_3": colorlist[3], 
                    "cNMF_4": colorlist[4], "cNMF_5": colorlist[6]}

colorlist = palettable.colorbrewer.sequential.Greens_9.mpl_colors
colormapping_myeloid = {"TAM1": colorlist[1], "DC": colorlist[2], 
                        "Mast": colorlist[4], "TAM2": colorlist[5]}

colorlist = palettable.colorbrewer.sequential.Oranges_5.mpl_colors
colormapping_fibroblast = {"Inflammatory CAF": colorlist[1], "Adipose CAF": colorlist[3], 
                           "Fibroblast": colorlist[4]}

colormapping_normal = (colormapping_myeloid | colormapping_fibroblast)

In [ ]:
color_mapping_idx = return_color(list_idx = heatmap_tumor_to_normal_df.index.to_numpy(), 
                             sigs = full_sigs, color_palette = colormapping_mal)
color_mapping_col = return_color(list_idx = heatmap_tumor_to_normal_df.columns.to_numpy(), 
                             sigs = normal_sigs, color_palette = colormapping_normal)

fig = plot_interaction_heatmap(heatmap_tumor_to_normal_df, color_mapping_idx, color_mapping_col, figsize=(2,1.5))
fig.savefig(figure_dir / patient_name / "tumor_to_normal_LIGREC.svg", dpi=200, bbox_inches="tight")

In [ ]:
color_mapping_idx = return_color(list_idx = heatmap_normal_to_tumor_df.index.to_numpy(), 
                             sigs = normal_sigs, color_palette = colormapping_normal)
color_mapping_col = return_color(list_idx = heatmap_normal_to_tumor_df.columns.to_numpy(), 
                             sigs = full_sigs, color_palette = colormapping_mal)

fig = plot_interaction_heatmap(heatmap_normal_to_tumor_df, color_mapping_idx, color_mapping_col, figsize=(2,1.5))
fig.savefig(figure_dir / patient_name / "normal_to_tumor_LIGREC.svg", dpi=200, bbox_inches="tight")

In [ ]:
adata.uns["Location refined_ligrec"]["means"].to_csv(visium_results_csv_dir / f"{patient_name}_LIGREC_means.csv")
adata.uns["Location refined_ligrec"]["pvalues"].to_csv(visium_results_csv_dir / f"{patient_name}_LIGREC_pvalues.csv")

In [ ]:
sc.pp.log1p(adata)

In [ ]:
LR_list = ["TGFBI","ITGB1","FN1","ITGA4","FN1","ITGA2","APP","TNFRSF21",]
LR_color_list = [colormapping_mal["cNMF_3"],colormapping_fibroblast["Adipose CAF"],
                colormapping_mal["cNMF_3"],colormapping_myeloid["DC"],
                colormapping_fibroblast["Adipose CAF"],colormapping_mal["cNMF_4"],
                colormapping_myeloid["TAM2"],colormapping_mal["cNMF_4"],]

fig, ax = plt.subplots(4,2, figsize=(7,8))
flatax = ax.flatten()
sq.pl.spatial_scatter(adata, color=LR_list, 
                      crop_coord=(y_min, x_min, y_max, x_max), 
                      alpha=1, ncols=2, 
                      fig=fig, ax=flatax)
for i in range(len(flatax)):
    flatax[i].set_title(LR_list[i], c=LR_color_list[i])
fig.tight_layout()
fig.savefig(figure_dir / patient_name / "spatial_rep_LR_LIGREC_full.svg", dpi=200, bbox_inches="tight")

# EGSFR1938_B

In [ ]:
patient_name = "EGSFR1938_B"

In [ ]:
resdir = cell2location_results_dir / patient_name 

sample_path = resdir / patient_name

adata = sc.read_h5ad(resdir / "cell2location_map" / "sp.h5ad")

tissue_path = spatial_dir / patient_name / "spatial/tissue_positions_list.csv"
tissue_position = pd.read_csv(tissue_path,index_col=0)
tissue_position = tissue_position.loc[adata.obs_names]

#Set coordinates
x_array=tissue_position["array_row"].tolist()
y_array=tissue_position["array_col"].tolist()
x_pixel=tissue_position["pxl_row_in_fullres"].tolist()
y_pixel=tissue_position["pxl_col_in_fullres"].tolist()

x_min, x_max = np.min(x_pixel), np.max(x_pixel)
y_min, y_max = np.min(y_pixel), np.max(y_pixel)

cancer_spots = get_cancer_spot(adata=adata)

cancer_spot = pd.DataFrame(np.zeros(adata.shape[0]),index=adata.obs_names,columns=["Cancer spot"])
cancer_spot.loc[adata.obs_names[cancer_spots],"Cancer spot"] = 1

adata.obs["Cancer spot"] = cancer_spot.replace({0: "Normal", 1: "Tumor"})

# Get carcinoma-specific expression and save in layers

In [ ]:
cell2loc_res = pd.read_csv(cell2location_results_dir / patient_name / "cell2location_map" / "celltype_abundance.csv",index_col=0)

# Get CNV profile

In [ ]:
adata.var = pd.concat([adata.var,gencode_df.loc[adata.var_names.intersection(df.index)]],axis=1)

In [ ]:
cnv.tl.infercnv(
    adata,
    reference_key="Cancer spot",
    reference_cat=[
        "Normal",
    ],
    window_size=250,
)

cnv.tl.pca(adata)
cnv.pp.neighbors(adata)
cnv.tl.leiden(adata, resolution=0.2)

In [ ]:
cnv.pl.chromosome_heatmap(adata, groupby="cnv_leiden", dendrogram=True)

In [ ]:
cnv.tl.umap(adata)
cnv.tl.cnv_score(adata)

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(11, 11))
ax4.axis("off")
cnv.pl.umap(
    adata,
    color="cnv_leiden",
    legend_loc="on data",
    legend_fontoutline=2,
    ax=ax1,
    show=False,
)
cnv.pl.umap(adata, color="cnv_score", ax=ax2, show=False)
cnv.pl.umap(adata, color="Cancer spot", ax=ax3)

In [ ]:
adata.obs["CNV_assignment"] = adata.obs.cnv_leiden.replace({'0': "Tumor", '1': "Normal", 
                                                            '2': "Mixed", '3': "Normal"})

adata.obs[["Cancer spot","cnv_leiden","cnv_score","CNV_assignment"]].to_csv(f"{patient_name}_cnv_info.csv")

# Carcinoma component normalize

In [ ]:
adata.layers["counts"] = adata.X.copy()
adata.X = adata.layers["Carcinoma"]

sc.pp.normalize_total(adata, target_sum=10000)
sc.pp.log1p(adata)

gex = adata.to_df()
gex = (gex - gex.mean())/gex.std()

for sig, genes in full_sigs.items():
    adata.obs[sig] = gex[gex.columns.intersection(genes)].mean(axis=1)

In [ ]:
smoothed_labels = get_full_smoothed_assignment(adata)

adata.obs["Smoothed label"] = smoothed_labels
adata.obs["Smoothed label"] = adata.obs["Smoothed label"].astype("category")

ass_col = "Smoothed label"

closest = pd.DataFrame([1000]*adata.obs.shape[0],index=adata.obs.index,columns=["Dist. to edge"])

closest.loc[adata.obs[ass_col]=="Mixed"] = 0
closest.loc[adata.obs[ass_col]=="Normal"] = -1

for n_rings in np.arange(1,11):
    closest = update_closest(adata, n_rings=n_rings, closest=closest, ass_col=ass_col)

adata.obs["Dist. to edge"] = closest

order_filtered = sorted(closest.value_counts()[(closest.value_counts()>=5)].index.get_level_values(0))

fig, ax = plt.subplots(2,3,figsize=(10,5))
flatax = ax.flatten()

for i,prog in enumerate([f"cNMF_{i}" for i in range(1,6)]):
    sns.barplot(data=adata.obs, x="Dist. to edge", y=prog, ax=flatax[i], order=order_filtered)
    pretty_ax(flatax[i])
    labels = flatax[i].get_xticklabels()
    labels = ["Normal" if l.get_text()=="-1" else ("Mixed" if l.get_text()=="0" else f"d={l.get_text()}") for l in labels]
    flatax[i].set_xticks(flatax[i].get_xticks(), labels,rotation=45,ha="right")
    flatax[i].set_xlabel("")
    flatax[i].set_ylabel(f"cNMF$_{i+1}$")
flatax[-1].axis("off")
fig.tight_layout()
fig.savefig(figure_dir / patient_name / "cnmf_distance_to_center.svg", dpi=200, bbox_inches="tight")

In [ ]:
adata.obs[["Dist. to edge","cNMF_1","cNMF_2","cNMF_3","cNMF_4","cNMF_5"]].to_csv(visium_results_csv_dir / f"{patient_name}_cNMF_dist_to_edge.csv")
pd.Series(order_filtered).to_csv(visium_results_csv_dir / f"{patient_name}_dist_to_edge_order.csv")

In [ ]:
fig, ax = plt.subplots(2,3, figsize=(10,5))
flatax = ax.flatten()
sq.pl.spatial_scatter(adata, color=[f"cNMF_{i}" for i in range(1,6)]+["Smoothed label"], 
                      crop_coord=(y_min, x_min, y_max, x_max), 
                      alpha=1, ncols=3, 
                      vcenter=0, vmin=-0.3, 
                      vmax=0.3, fig=fig, ax=flatax)
for i in range(len(flatax)-1):
    flatax[i].set_title(f"cNMF$_{i+1}$")
flatax[-1].set_title("CNV-derived spot label")
fig.tight_layout()
fig.savefig(figure_dir / patient_name / "scatterplot_spatial.png", dpi=200, bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots(2,3,figsize=(7,3.5))
flatax = ax.flatten()
list_ct = ["Carcinoma","Fibroblast","Endothelial","Myeloid","Lymphoid","Muscle"]
for i in range(len(list_ct)):
    sns.boxplot(data=adata.obs, x="Smoothed label", y=list_ct[i], ax=flatax[i], order=["Normal","Mixed","Tumor"])
    pretty_ax(flatax[i])
    flatax[i].set_ylabel("N cells")
    flatax[i].set_xlabel("")
    flatax[i].set_title(f"{list_ct[i]}")
    flatax[i].set_xticks(flatax[i].get_xticks(), flatax[i].get_xticklabels(), rotation=45, ha="right")
    
fig.tight_layout()
fig.savefig(figure_dir / patient_name / "N_cells_perspot.png", dpi=200, bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots(2,3,figsize=(7,3.5))
flatax = ax.flatten()

pairs = [("Normal","Tumor"),("Normal","Mixed"),("Mixed","Tumor")]
for i,prog in enumerate([f"cNMF_{i}" for i in range(1,6)]):
    sns.boxplot(data=adata.obs, x="Smoothed label", y=prog, ax=flatax[i], order=["Normal","Mixed","Tumor"])
    pretty_ax(flatax[i])
    flatax[i].set_xticks(flatax[i].get_xticks(),flatax[i].get_xticklabels(),rotation=45,ha="right")
    flatax[i].set_xlabel("")
    flatax[i].set_ylabel(f"cNMF$_{i+1}$")
    annotator = Annotator(flatax[i], pairs, data=adata.obs, x="Smoothed label", 
                          y=prog, order=["Normal","Mixed","Tumor"])
    annotator.configure(test='Mann-Whitney', text_format='simple', show_test_name=False, loc='inside')
    annotator.apply_and_annotate()
flatax[-1].axis("off")
fig.tight_layout()
fig.savefig(figure_dir / patient_name / "dist_cnmf_cnv_assignemnt.png", dpi=200, bbox_inches="tight")

In [ ]:
location_refined = adata.obs[[ass_col]].copy().astype(str)
location_refined.columns = ["Location refined"]

adata.obs["Location refined"] = get_refined_location(adata=adata, n_rings=1, 
                                                     location_refined=location_refined, ass_col="Smoothed label")
adata.obs["Location refined"] = adata.obs["Location refined"].astype("category")

# Run LIGREC

In [ ]:
adata.X = adata.layers["counts"].copy()
sc.pp.normalize_total(adata, target_sum=10000)

In [ ]:
sq.gr.ligrec(
    adata, use_raw=False, interactions=df_interaction,
    n_perms=500, corr_method="fdr_bh",
    cluster_key="Location refined",
)

tumor_per_interactions = adata.uns["Location refined_ligrec"]["pvalues"]["Tumor_Periphery"]

tumor_per_means = adata.uns["Location refined_ligrec"]["means"]["Tumor_Periphery"]

tumor_core_interactions = adata.uns["Location refined_ligrec"]["pvalues"]["Tumor_Core"]

tumor_core_means = adata.uns["Location refined_ligrec"]["means"]["Tumor_Core"]

normal_interactions = adata.uns["Location refined_ligrec"]["pvalues"]["Normal_Periphery"]

normal_means = adata.uns["Location refined_ligrec"]["means"]["Normal_Periphery"]

In [ ]:
sign_int = tumor_core_interactions["Tumor_Core"][tumor_core_interactions["Tumor_Core"]<0.05]

heatmap_tumor_df = get_heatmap_interactions_tumor(sign_int=sign_int, full_sigs=full_sigs, tumor_means=tumor_core_means)

sign_int = tumor_per_interactions["Normal_Periphery"][tumor_per_interactions["Normal_Periphery"]<0.05]

heatmap_tumor_to_normal_df = get_heatmap_interactions_tumor_to_normal(sign_int=sign_int, full_sigs=full_sigs, 
                                                             normal_sigs=normal_sigs, tumor_means=tumor_per_means)

sign_int = normal_interactions["Tumor_Periphery"][normal_interactions["Tumor_Periphery"]<0.05]

heatmap_normal_to_tumor_df = get_heatmap_interactions_normal_to_tumor(sign_int=sign_int, full_sigs=full_sigs, 
                                                             normal_sigs=normal_sigs, normal_means=normal_means)

In [ ]:
import palettable
colorlist = palettable.colorbrewer.qualitative.Set1_7.mpl_colors
colormapping_mal = {"cNMF_1": colorlist[0], "cNMF_2": colorlist[1], "cNMF_3": colorlist[3], 
                    "cNMF_4": colorlist[4], "cNMF_5": colorlist[6]}

colorlist = palettable.colorbrewer.sequential.Greens_9.mpl_colors
colormapping_myeloid = {"TAM1": colorlist[1], "DC": colorlist[2], 
                        "Mast": colorlist[4], "TAM2": colorlist[5]}

colorlist = palettable.colorbrewer.sequential.Oranges_5.mpl_colors
colormapping_fibroblast = {"Inflammatory CAF": colorlist[1], "Adipose CAF": colorlist[3], 
                           "Fibroblast": colorlist[4]}

colormapping_normal = (colormapping_myeloid | colormapping_fibroblast)

In [ ]:
color_mapping_idx = return_color(list_idx = heatmap_tumor_to_normal_df.index.to_numpy(), 
                             sigs = full_sigs, color_palette = colormapping_mal)
color_mapping_col = return_color(list_idx = heatmap_tumor_to_normal_df.columns.to_numpy(), 
                             sigs = normal_sigs, color_palette = colormapping_normal)

fig = plot_interaction_heatmap(heatmap_tumor_to_normal_df, color_mapping_idx, color_mapping_col, figsize=(2,1.5))
fig.savefig(figure_dir / patient_name / "tumor_to_normal_LIGREC.svg", dpi=200, bbox_inches="tight")

In [ ]:
color_mapping_idx = return_color(list_idx = heatmap_normal_to_tumor_df.index.to_numpy(), 
                             sigs = normal_sigs, color_palette = colormapping_normal)
color_mapping_col = return_color(list_idx = heatmap_normal_to_tumor_df.columns.to_numpy(), 
                             sigs = full_sigs, color_palette = colormapping_mal)

fig = plot_interaction_heatmap(heatmap_normal_to_tumor_df, color_mapping_idx, color_mapping_col, figsize=(2,1.5))
fig.savefig(figure_dir / patient_name / "normal_to_tumor_LIGREC.svg", dpi=200, bbox_inches="tight")

In [ ]:
adata.uns["Location refined_ligrec"]["means"].to_csv(visium_results_csv_dir / f"{patient_name}_LIGREC_means.csv")
adata.uns["Location refined_ligrec"]["pvalues"].to_csv(visium_results_csv_dir / f"{patient_name}_LIGREC_pvalues.csv")

# EGSFR1938_C

In [ ]:
patient_name = "EGSFR1938_C"

In [ ]:
resdir = cell2location_results_dir / patient_name 

sample_path = resdir / patient_name

adata = sc.read_h5ad(resdir / "cell2location_map" / "sp.h5ad")

tissue_path = spatial_dir / patient_name / "spatial/tissue_positions_list.csv"
tissue_position = pd.read_csv(tissue_path,index_col=0)
tissue_position = tissue_position.loc[adata.obs_names]

#Set coordinates
x_array=tissue_position["array_row"].tolist()
y_array=tissue_position["array_col"].tolist()
x_pixel=tissue_position["pxl_row_in_fullres"].tolist()
y_pixel=tissue_position["pxl_col_in_fullres"].tolist()

x_min, x_max = np.min(x_pixel), np.max(x_pixel)
y_min, y_max = np.min(y_pixel), np.max(y_pixel)

cancer_spots = get_cancer_spot(adata=adata)

cancer_spot = pd.DataFrame(np.zeros(adata.shape[0]),index=adata.obs_names,columns=["Cancer spot"])
cancer_spot.loc[adata.obs_names[cancer_spots],"Cancer spot"] = 1

adata.obs["Cancer spot"] = cancer_spot.replace({0: "Normal", 1: "Tumor"})

# Get carcinoma-specific expression and save in layers

In [ ]:
cell2loc_res = pd.read_csv(cell2location_results_dir / patient_name / "cell2location_map" / "celltype_abundance.csv",index_col=0)

# Get CNV profile

In [ ]:
adata.var = pd.concat([adata.var,gencode_df.loc[adata.var_names.intersection(df.index)]],axis=1)

In [ ]:
cnv.tl.infercnv(
    adata,
    reference_key="Cancer spot",
    reference_cat=[
        "Normal",
    ],
    window_size=250,
)

cnv.tl.pca(adata)
cnv.pp.neighbors(adata)
cnv.tl.leiden(adata, resolution=0.2)

In [ ]:
cnv.pl.chromosome_heatmap(adata, groupby="cnv_leiden", dendrogram=True)

In [ ]:
cnv.tl.umap(adata)
cnv.tl.cnv_score(adata)

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(11, 11))
ax4.axis("off")
cnv.pl.umap(
    adata,
    color="cnv_leiden",
    legend_loc="on data",
    legend_fontoutline=2,
    ax=ax1,
    show=False,
)
cnv.pl.umap(adata, color="cnv_score", ax=ax2, show=False)
cnv.pl.umap(adata, color="Cancer spot", ax=ax3)

In [ ]:
adata.obs["CNV_assignment"] = adata.obs.cnv_leiden.replace({'0': "Tumor", '2': "Mixed", 
                                                            '1': "Normal", '3': "Normal"})

adata.obs[["Cancer spot","cnv_leiden","cnv_score","CNV_assignment"]].to_csv(f"{patient_name}_cnv_info.csv")

# Carcinoma component normalize

In [ ]:
adata.layers["counts"] = adata.X.copy()
adata.X = adata.layers["Carcinoma"]

sc.pp.normalize_total(adata, target_sum=10000)
sc.pp.log1p(adata)

gex = adata.to_df()
gex = (gex - gex.mean())/gex.std()

for sig, genes in full_sigs.items():
    adata.obs[sig] = gex[gex.columns.intersection(genes)].mean(axis=1)

In [ ]:
smoothed_labels = get_full_smoothed_assignment(adata)

adata.obs["Smoothed label"] = smoothed_labels
adata.obs["Smoothed label"] = adata.obs["Smoothed label"].astype("category")

ass_col = "Smoothed label"

closest = pd.DataFrame([1000]*adata.obs.shape[0],index=adata.obs.index,columns=["Dist. to edge"])

closest.loc[adata.obs[ass_col]=="Mixed"] = 0
closest.loc[adata.obs[ass_col]=="Normal"] = -1

for n_rings in np.arange(1,11):
    closest = update_closest(adata, n_rings=n_rings, closest=closest, ass_col=ass_col)

adata.obs["Dist. to edge"] = closest

order_filtered = sorted(closest.value_counts()[(closest.value_counts()>=5)].index.get_level_values(0))

fig, ax = plt.subplots(2,3,figsize=(10,5))
flatax = ax.flatten()

for i,prog in enumerate([f"cNMF_{i}" for i in range(1,6)]):
    sns.barplot(data=adata.obs, x="Dist. to edge", y=prog, ax=flatax[i], order=order_filtered)
    pretty_ax(flatax[i])
    labels = flatax[i].get_xticklabels()
    labels = ["Normal" if l.get_text()=="-1" else ("Mixed" if l.get_text()=="0" else f"d={l.get_text()}") for l in labels]
    flatax[i].set_xticks(flatax[i].get_xticks(), labels,rotation=45,ha="right")
    flatax[i].set_xlabel("")
    flatax[i].set_ylabel(f"cNMF$_{i+1}$")
flatax[-1].axis("off")
fig.tight_layout()
fig.savefig(figure_dir / patient_name / "cnmf_distance_to_center.svg", dpi=200, bbox_inches="tight")

In [ ]:
adata.obs[["Dist. to edge","cNMF_1","cNMF_2","cNMF_3","cNMF_4","cNMF_5"]].to_csv(visium_results_csv_dir / f"{patient_name}_cNMF_dist_to_edge.csv")
pd.Series(order_filtered).to_csv(visium_results_csv_dir / f"{patient_name}_dist_to_edge_order.csv")

In [ ]:
fig, ax = plt.subplots(2,3, figsize=(10,5))
flatax = ax.flatten()
sq.pl.spatial_scatter(adata, color=[f"cNMF_{i}" for i in range(1,6)]+["Smoothed label"], 
                      crop_coord=(y_min, x_min, y_max, x_max), 
                      alpha=1, ncols=3, 
                      vcenter=0, vmin=-0.3, 
                      vmax=0.3, fig=fig, ax=flatax)
for i in range(len(flatax)-1):
    flatax[i].set_title(f"cNMF$_{i+1}$")
flatax[-1].set_title("CNV-derived spot label")
fig.tight_layout()
fig.savefig(figure_dir / patient_name / "scatterplot_spatial.png", dpi=200, bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots(2,3,figsize=(7,3.5))
flatax = ax.flatten()
list_ct = ["Carcinoma","Fibroblast","Endothelial","Myeloid","Lymphoid","Muscle"]
for i in range(len(list_ct)):
    sns.boxplot(data=adata.obs, x="Smoothed label", y=list_ct[i], ax=flatax[i], order=["Normal","Mixed","Tumor"])
    pretty_ax(flatax[i])
    flatax[i].set_ylabel("N cells")
    flatax[i].set_xlabel("")
    flatax[i].set_title(f"{list_ct[i]}")
    flatax[i].set_xticks(flatax[i].get_xticks(), flatax[i].get_xticklabels(), rotation=45, ha="right")
    #flatax[i].set_ylim(0,4)
fig.tight_layout()
fig.savefig(figure_dir / patient_name / "N_cells_perspot.png", dpi=200, bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots(2,3,figsize=(7,3.5))
flatax = ax.flatten()

pairs = [("Normal","Tumor"),("Normal","Mixed"),("Mixed","Tumor")]
for i,prog in enumerate([f"cNMF_{i}" for i in range(1,6)]):
    sns.boxplot(data=adata.obs, x="Smoothed label", y=prog, ax=flatax[i], order=["Normal","Mixed","Tumor"])
    pretty_ax(flatax[i])
    flatax[i].set_xticks(flatax[i].get_xticks(),flatax[i].get_xticklabels(),rotation=45,ha="right")
    flatax[i].set_xlabel("")
    flatax[i].set_ylabel(f"cNMF$_{i+1}$")
    annotator = Annotator(flatax[i], pairs, data=adata.obs, x="Smoothed label", 
                          y=prog, order=["Normal","Mixed","Tumor"])
    annotator.configure(test='Mann-Whitney', text_format='simple', show_test_name=False, loc='inside')
    annotator.apply_and_annotate()
flatax[-1].axis("off")
fig.tight_layout()
fig.savefig(figure_dir / patient_name / "dist_cnmf_cnv_assignemnt.png", dpi=200, bbox_inches="tight")

In [ ]:
location_refined = adata.obs[[ass_col]].copy().astype(str)
location_refined.columns = ["Location refined"]

adata.obs["Location refined"] = get_refined_location(adata=adata, n_rings=1, 
                                                     location_refined=location_refined, ass_col="Smoothed label")
adata.obs["Location refined"] = adata.obs["Location refined"].astype("category")

# Run LIGREC

In [ ]:
adata.X = adata.layers["counts"].copy()
sc.pp.normalize_total(adata, target_sum=10000)

In [ ]:
sq.gr.ligrec(
    adata, use_raw=False, interactions=df_interaction,
    n_perms=500, corr_method="fdr_bh",
    cluster_key="Location refined",
)

tumor_per_interactions = adata.uns["Location refined_ligrec"]["pvalues"]["Tumor_Periphery"]

tumor_per_means = adata.uns["Location refined_ligrec"]["means"]["Tumor_Periphery"]

tumor_core_interactions = adata.uns["Location refined_ligrec"]["pvalues"]["Tumor_Core"]

tumor_core_means = adata.uns["Location refined_ligrec"]["means"]["Tumor_Core"]

normal_interactions = adata.uns["Location refined_ligrec"]["pvalues"]["Normal_Periphery"]

normal_means = adata.uns["Location refined_ligrec"]["means"]["Normal_Periphery"]

In [ ]:
sign_int = tumor_core_interactions["Tumor_Core"][tumor_core_interactions["Tumor_Core"]<0.05]

heatmap_tumor_df = get_heatmap_interactions_tumor(sign_int=sign_int, full_sigs=full_sigs, tumor_means=tumor_core_means)

sign_int = tumor_per_interactions["Normal_Periphery"][tumor_per_interactions["Normal_Periphery"]<0.05]

heatmap_tumor_to_normal_df = get_heatmap_interactions_tumor_to_normal(sign_int=sign_int, full_sigs=full_sigs, 
                                                             normal_sigs=normal_sigs, tumor_means=tumor_per_means)

sign_int = normal_interactions["Tumor_Periphery"][normal_interactions["Tumor_Periphery"]<0.05]

heatmap_normal_to_tumor_df = get_heatmap_interactions_normal_to_tumor(sign_int=sign_int, full_sigs=full_sigs, 
                                                             normal_sigs=normal_sigs, normal_means=normal_means)

In [ ]:
import palettable
colorlist = palettable.colorbrewer.qualitative.Set1_7.mpl_colors
colormapping_mal = {"cNMF_1": colorlist[0], "cNMF_2": colorlist[1], "cNMF_3": colorlist[3], 
                    "cNMF_4": colorlist[4], "cNMF_5": colorlist[6]}

colorlist = palettable.colorbrewer.sequential.Greens_9.mpl_colors
colormapping_myeloid = {"TAM1": colorlist[1], "DC": colorlist[2], 
                        "Mast": colorlist[4], "TAM2": colorlist[5]}

colorlist = palettable.colorbrewer.sequential.Oranges_5.mpl_colors
colormapping_fibroblast = {"Inflammatory CAF": colorlist[1], "Adipose CAF": colorlist[3], 
                           "Fibroblast": colorlist[4]}

colormapping_normal = (colormapping_myeloid | colormapping_fibroblast)

In [ ]:
color_mapping_idx = return_color(list_idx = heatmap_tumor_to_normal_df.index.to_numpy(), 
                             sigs = full_sigs, color_palette = colormapping_mal)
color_mapping_col = return_color(list_idx = heatmap_tumor_to_normal_df.columns.to_numpy(), 
                             sigs = normal_sigs, color_palette = colormapping_normal)

fig = plot_interaction_heatmap(heatmap_tumor_to_normal_df, color_mapping_idx, color_mapping_col, figsize=(2,1.5))
fig.savefig(figure_dir / patient_name / "tumor_to_normal_LIGREC.svg", dpi=200, bbox_inches="tight")

In [ ]:
color_mapping_idx = return_color(list_idx = heatmap_normal_to_tumor_df.index.to_numpy(), 
                             sigs = normal_sigs, color_palette = colormapping_normal)
color_mapping_col = return_color(list_idx = heatmap_normal_to_tumor_df.columns.to_numpy(), 
                             sigs = full_sigs, color_palette = colormapping_mal)

fig = plot_interaction_heatmap(heatmap_normal_to_tumor_df, color_mapping_idx, color_mapping_col, figsize=(2,1.5))
fig.savefig(figure_dir / patient_name / "normal_to_tumor_LIGREC.svg", dpi=200, bbox_inches="tight")

In [ ]:
adata.uns["Location refined_ligrec"]["means"].to_csv(visium_results_csv_dir / f"{patient_name}_LIGREC_means.csv")
adata.uns["Location refined_ligrec"]["pvalues"].to_csv(visium_results_csv_dir / f"{patient_name}_LIGREC_pvalues.csv")

# Full dist to edge plot

In [ ]:
patients = ["EGSFR1938_A","EGSFR1938_B","EGSFR1938_C","EGSFR0074_A","EGSFR0148"]

patient_id_mapping = {"EGSFR0074_A": "P4", 
                      "EGSFR0148": "P5", "EGSFR1938_A": "P8_A", 
                      "EGSFR1938_B": "P8_B", "EGSFR1938_C": "P8_C"}

In [ ]:
all_scores = []
for patient_name in patients:
    scores = pd.read_csv(visium_results_csv_dir / f"{patient_name}_cNMF_dist_to_edge.csv", index_col=0)
    order = pd.read_csv(visium_results_csv_dir / f"{patient_name}_dist_to_edge_order.csv", index_col=0).values.ravel()

    scores = scores[scores["Dist. to edge"].isin(order)].sort_values('Dist. to edge')
    scores["Patient"] = patient_id_mapping[patient_name]
    
    all_scores.append(scores)
all_scores = pd.concat(all_scores)

In [ ]:
fig, ax = plt.subplots(3,2,figsize=(11,4.6))
flatax = ax.flatten()
for i,program in enumerate([f"cNMF_{i}" for i in range(1,6)]):
    sns.barplot(data=all_scores, x="Patient", y=program, hue="Dist. to edge",ax=flatax[i])
    if i==3:
        flatax[i].legend(frameon=False, title="Dist. to edge",bbox_to_anchor=(1,1,0,0))
    else:
        flatax[i].legend().set_visible(False)
    pretty_ax(flatax[i])
    flatax[i].set_ylabel(f"cNMF$_{i+1}$")
flatax[-1].axis("off")
fig.savefig(figure_dir / "full_dist_to_edge_cNMF.svg", dpi=200, bbox_inches="tight")

# Full LIGREC interactions

In [ ]:
def format_p_val(p: float) -> str:
    if p>0.1:
        return "ns"
    elif 0.01<p<=0.1:
        return "*"
    elif 0.001<p<=0.01:
        return "**"
    else:
        return "***"

In [ ]:
import palettable
colorlist = palettable.colorbrewer.qualitative.Set1_7.mpl_colors
colormapping_mal = {"cNMF_1": colorlist[0], "cNMF_2": colorlist[1], "cNMF_3": colorlist[3], 
                    "cNMF_4": colorlist[4], "cNMF_5": colorlist[6]}

colorlist = palettable.colorbrewer.sequential.Greens_9.mpl_colors
colormapping_myeloid = {"TAM1": colorlist[1], "DC": colorlist[2], 
                        "Mast": colorlist[4], "TAM2": colorlist[5]}

colorlist = palettable.colorbrewer.sequential.Oranges_5.mpl_colors
colormapping_fibroblast = {"Inflammatory CAF": colorlist[1], "Adipose CAF": colorlist[3], 
                           "Fibroblast": colorlist[4]}

colormapping_normal = (colormapping_myeloid | colormapping_fibroblast)

In [ ]:
patients = ["EGSFR1938_A","EGSFR1938_B","EGSFR1938_C","EGSFR0074_A","EGSFR0148"]

patient_id_mapping = {"EGSFR0074_A": "P4", 
                      "EGSFR0148": "P5", "EGSFR1938_A": "P8_A", 
                      "EGSFR1938_B": "P8_B", "EGSFR1938_C": "P8_C"}

In [ ]:
normal_to_tumor_source = ["FN1","ITGB1","FYN","VCAN","APP"]
normal_to_tumor_target = ["RACK1","BIRC5","FN1","ITGA2","TNFRSF21",]

tumor_to_normal_source = ["ACTB","TGFBI","VCAN","SLIT2","FN1","FBN1","THBS2","JUP","NTN4"]
tumor_to_normal_target = ["ITGB1","FN1","ITGA4","FER","TTN","APP"]

normal_to_tumor_interactions = {pat: pd.DataFrame(np.zeros((len(normal_to_tumor_source),len(normal_to_tumor_target))), 
             index=normal_to_tumor_source, columns=normal_to_tumor_target) for pat in patients}
tumor_to_normal_interactions = {pat: pd.DataFrame(np.zeros((len(tumor_to_normal_source),len(tumor_to_normal_target))), 
             index=tumor_to_normal_source, columns=tumor_to_normal_target) for pat in patients}


normal_to_tumor_pvalues = {pat: pd.DataFrame(np.zeros((len(normal_to_tumor_source),len(normal_to_tumor_target))), 
             index=normal_to_tumor_source, columns=normal_to_tumor_target) for pat in patients}
tumor_to_normal_pvalues = {pat: pd.DataFrame(np.zeros((len(tumor_to_normal_source),len(tumor_to_normal_target))), 
             index=tumor_to_normal_source, columns=tumor_to_normal_target) for pat in patients}


all_means, all_ps = {},{}
for patient_name in patients:
    all_means[patient_name] = pd.read_csv(visium_results_csv_dir / f"{patient_name}_LIGREC_means.csv", 
                                          index_col=[0,1], header=[0, 1])
    all_ps[patient_name] = pd.read_csv(visium_results_csv_dir / f"{patient_name}_LIGREC_pvalues.csv", 
                                       index_col=[0,1], header=[0, 1])

In [ ]:
for patient_name in patients:
    df1 = all_means[patient_name]["Normal_Periphery"]["Tumor_Periphery"].reset_index()
    df2 = all_ps[patient_name]["Normal_Periphery"]["Tumor_Periphery"].reset_index()
    for source in normal_to_tumor_source:
        for target in normal_to_tumor_target:
            found = df1[(df1.source==source) & (df1.target==target)]
            pfound = df2[(df2.source==source) & (df2.target==target)]
            if found.shape[0]==0:
                normal_to_tumor_interactions[patient_name].loc[source,target]=np.nan
                normal_to_tumor_pvalues[patient_name].loc[source,target]=1
            else:
                normal_to_tumor_interactions[patient_name].loc[source,target]=found["Tumor_Periphery"].values[0]
                normal_to_tumor_pvalues[patient_name].loc[source,target]=pfound["Tumor_Periphery"].values[0]
    normal_to_tumor_pvalues[patient_name] = normal_to_tumor_pvalues[patient_name].applymap(format_p_val)

color_mapping_idx = return_color(list_idx = normal_to_tumor_interactions[patients[0]].index.to_numpy(), 
                             sigs = normal_sigs, color_palette = colormapping_normal)
color_mapping_col = return_color(list_idx = normal_to_tumor_interactions[patients[0]].columns.to_numpy(), 
                             sigs = full_sigs, color_palette = colormapping_mal)

fig, ax = plt.subplots(1,5, figsize=(12,1.5))
flatax = ax.flatten()
for i in range(len(normal_to_tumor_interactions)):
    
    sns.heatmap(data=normal_to_tumor_interactions[patients[i]], ax=flatax[i], cmap="vlag", 
                center=0, vmin=0, vmax=15, annot=normal_to_tumor_pvalues[patients[i]], fmt="")
    for j,tick in enumerate(flatax[i].get_yticklabels()):
        tick.set_color(color_mapping_idx[tick.get_text()])
    for j,tick in enumerate(flatax[i].get_xticklabels()):
        tick.set_color(color_mapping_col[tick.get_text()])
    
    if i>0:
        flatax[i].get_yaxis().set_visible(False)
fig.savefig(figure_dir / "full_LIGREC_normal_to_tumor.svg", dpi=200, bbox_inches="tight")

In [ ]:
for patient_name in patients:
    df1 = all_means[patient_name]["Tumor_Periphery"]["Normal_Periphery"].reset_index()
    df2 = all_ps[patient_name]["Tumor_Periphery"]["Normal_Periphery"].reset_index()
    for source in tumor_to_normal_source:
        for target in tumor_to_normal_target:
            found = df1[(df1.source==source) & (df1.target==target)]
            pfound = df2[(df2.source==source) & (df2.target==target)]
            if found.shape[0]==0:
                tumor_to_normal_interactions[patient_name].loc[source,target]=np.nan
                tumor_to_normal_pvalues[patient_name].loc[source,target]=1
            else:
                tumor_to_normal_interactions[patient_name].loc[source,target]=found["Normal_Periphery"].values[0]
                tumor_to_normal_pvalues[patient_name].loc[source,target]=pfound["Normal_Periphery"].values[0]
    tumor_to_normal_pvalues[patient_name] = tumor_to_normal_pvalues[patient_name].applymap(format_p_val)

color_mapping_idx = return_color(list_idx = tumor_to_normal_interactions[patients[0]].index.to_numpy(), 
                             sigs = full_sigs, color_palette = colormapping_mal)
color_mapping_col = return_color(list_idx = tumor_to_normal_interactions[patients[0]].columns.to_numpy(), 
                             sigs = normal_sigs, color_palette = colormapping_normal)

fig, ax = plt.subplots(1,5, figsize=(12,2.5))
flatax = ax.flatten()
for i in range(len(tumor_to_normal_interactions)):
    
    sns.heatmap(data=tumor_to_normal_interactions[patients[i]], ax=flatax[i], cmap="vlag", 
                center=0, vmin=0, vmax=15, annot=tumor_to_normal_pvalues[patients[i]], fmt="")
    for j,tick in enumerate(flatax[i].get_yticklabels()):
        tick.set_color(color_mapping_idx[tick.get_text()])
    for j,tick in enumerate(flatax[i].get_xticklabels()):
        tick.set_color(color_mapping_col[tick.get_text()])
    
    if i>0:
        flatax[i].get_yaxis().set_visible(False)
fig.savefig(figure_dir / "full_LIGREC_tumor_to_normal.svg", dpi=200, bbox_inches="tight")